In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph","total_network"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs,total_lengths = results[inst]
        for i,angle in enumerate(angles):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                     "t": [ts[i]],
                     "hyph": [hyphs[i]],
                     "total_network" : [total_lengths[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)
corrected = infos.loc[(infos["straightness"] <= 1)&(infos["plate"] !=435)& (infos["speed"] >=25)& (infos["speed"] <400)]
plate_94 = corrected.loc[corrected ['plate']==94]


In [2]:
plate_94 = corrected.loc[(corrected ['plate']==94) & (corrected ['t']<=60)& (corrected ['speed']>=150)]
# plate_94 = corrected.loc[corrected ['plate']==94]


In [25]:
plate_94

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,t,hyph,total_network
698,94,"(39, 269, 329)",25*,-8.130102,-0.033951,1.339855,239.465595,239.465595,0.995367,0.000000,1,1.749240e+05
699,94,"(39, 269, 329)",25*,-9.019322,-0.042351,1.452899,212.967115,212.967115,0.990866,1.000000,1,1.832130e+05
700,94,"(39, 269, 329)",25*,-9.473778,-0.044439,1.346377,213.187075,213.187075,0.989844,2.000000,1,1.908655e+05
701,94,"(39, 269, 329)",25*,3.419983,0.016867,1.147826,202.766341,206.203059,0.995394,3.000000,1,1.994984e+05
704,94,"(39, 269, 329)",25*,-12.733252,-0.089661,0.583333,142.015624,142.015624,0.984545,5.983333,1,2.168602e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
2895,94,"(39, 269, 329)",25*,-12.985598,-0.050187,3.918841,258.746212,258.746212,0.995583,59.016667,14696,1.187199e+06
2896,94,"(39, 269, 329)",25*,-11.888658,-0.059135,4.150000,201.043765,201.043765,0.978070,58.016667,14807,1.160717e+06
2897,94,"(39, 269, 329)",25*,-16.437415,-0.079272,4.478986,207.355018,207.355018,0.990772,59.016667,14807,1.187199e+06
2898,94,"(39, 269, 329)",25*,-7.349167,-0.035690,2.278986,205.919338,205.919338,0.965981,58.016667,14840,1.160717e+06


In [3]:
baits_sort = plate_94.sort_values("t")
N=80
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
ax.set_xlim(0,30)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(plate_94["t"],plate_94['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["t"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["t"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["t"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [3]:
baits_sort = plate_94.sort_values("total_network")
N=80
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('total network($\mu m$)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(plate_94["total_network"],plate_94['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["total_network"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
160**2*np.log(2)**2/(2*1000**2)*24**2,56/20*np.log(2)/1000*24**2

(3.542283981016115, 1.1179077728070796)

In [11]:
160**2*np.log(2)**2/(2*1000**2)*24**2 + 3*np.log(2)/1000*24**2

4.7400423090237

In [39]:
np.sqrt(4*np.log(2)/1500*24**2)

1.0318304460230256

In [40]:
30000/3600

8.333333333333334

In [5]:
0.80*24

19.200000000000003